### This notebook is testing different sentiment signal strategies

In [170]:
import pandas as pd
import numpy as np

In [171]:
# importing the data
dcb_VADER = pd.read_csv('../Project_2/Sentiment_Scores/VADER/dcb_VADER_sentiment.csv', parse_dates=True, index_col="date", infer_datetime_format=True)
crsh_VADER = pd.read_csv('../Project_2/Sentiment_Scores/VADER/crsh_VADER_sentiment.csv', parse_dates=True, index_col="date", infer_datetime_format=True)
cvd_VADER = pd.read_csv('../Project_2/Sentiment_Scores/VADER/cvd_VADER_sentiment.csv', parse_dates=True, index_col="date", infer_datetime_format=True)

In [172]:
# turning the index into date format
dcb_VADER.index = pd.DatetimeIndex(dcb_VADER.index).date
crsh_VADER.index = pd.DatetimeIndex(crsh_VADER.index).date
cvd_VADER.index = pd.DatetimeIndex(cvd_VADER.index).date

In [173]:
# grouping each df by the date then taking the mean
dcb_VADER = dcb_VADER.groupby(by=dcb_VADER.index).mean()
crsh_VADER = crsh_VADER.groupby(by=crsh_VADER.index).mean()
cvd_VADER = cvd_VADER.groupby(by=cvd_VADER.index).mean()

In [133]:
# strategy 1: no boundries
def get_simple_signals(data):
    sentiment_signal = []
    for score in data['compound']:
        if score > 0:
            sentiment_signal.append(1)   
        elif score < 0:
            sentiment_signal.append(-1)
        else:
             sentiment_signal.append(0)
    data['sentiment signal'] = sentiment_signal 
    data.drop(columns='compound', inplace=True)
    export_name = data.index[0]
    data.to_csv(f'../Project_2/Sentiment_Signals/simple_{export_name}_signal.csv')

In [134]:
get_simple_signals(cvd_VADER)

In [ ]:
get_boundry_signals(dcb_VADER)
dcb_VADER.head()

,compound,signal
1997-06-01,-0.028627,0
1997-06-02,-0.381800,-1
1997-06-03,0.297325,1
1997-06-04,0.234550,1
1997-06-05,0.103150,0


In [179]:
pick_boundry_signals(cvd_VADER, 0.1, -0.3)

In [174]:
# strategy 3: comparing market sentiment to the 5 day rolling average and making a signal based on the diff

def get_std_signals(data):
    copy = data.copy()
    copy['diff'] = copy['compound'].diff()
    copy['rolling std'] = copy['compound'].rolling(window=5).std()
    copy.dropna(inplace=True)
    
    sentiment_signal = []
    for column, row in copy.iterrows():
        if row['diff'] > (2 * row['rolling std']):
            sentiment_signal.append(-1)
        elif row['diff'] > (0):
            sentiment_signal.append(1)
        elif row['diff'] < -(2 * row['rolling std']):
            sentiment_signal.append(1)
        elif row['diff'] < (0):
            sentiment_signal.append(-1)
        else:
            sentiment_signal.append(0)
    copy['signal'] = sentiment_signal
    
    copy.drop(columns=['diff','rolling std', 'compound'], inplace=True)
    export_name = copy.index[0]
    copy.to_csv(f'../Project_2/Sentiment_Signals/std_{export_name}_signal.csv')

In [178]:
get_std_signals(crsh_VADER)

### NOT IN USE

In [ ]:
# strategy 2: simple boundries

def get_boundry_signals(data):
    sentiment_signal = []
    for score in data['compound']:
        if score > 0.2:
            sentiment_signal.append(1)   
        elif score < 0.2 and score > -0.2:
            sentiment_signal.append(0)
        elif score < -0.2:
            sentiment_signal.append(-1)
    data['signal'] = sentiment_signal

In [ ]:
# stategy 2.1: pick your boundry signals

def pick_boundry_signals(data, upper_bound, lower_bound):
    sentiment_signal = []
    for score in data['compound']:
        if score > upper_bound:
            sentiment_signal.append(1)   
        elif score < upper_bound and score > lower_bound:
            sentiment_signal.append(0)
        elif score < lower_bound:
            sentiment_signal.append(-1)
    data['signal'] = sentiment_signal